In [1]:
# Before beginning, I did research on how best to deal with the incredibly imbalanced dataset and different sampling methods.
# I initially did my modeling in RapidMiner, but found that I wasn't getting good results that generalized well when I submitted
# to Kaggle. I also didn't think the algorithms I was using were doing that well.
# I also did research on which models are best for this kind of data, and came across some boosting algorithms that would work
# efficiently with high performance generalization, so I switched to Python. I chose to try to optimize various versions of 
# XGBoost, CatBoost, and LightGBM, and tried various ensembling methods. I found that my best results came from an ensemble
# of my best of each of the three types of models, using a threshold of 0.6.

In [2]:
# I used the following resources/tutorials about parameter optimization and XGBoost, CatBoost, and LightGBM:
# https://github.com/catboost/tutorials/blob/master/classification/classification_with_parameter_tuning_tutorial.ipynb
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
# https://medium.com/analytics-vidhya/hyperparameters-optimization-for-lightgbm-catboost-and-xgboost-regressors-using-bayesian-6e7c495947a9
# https://github.com/catboost/catboost

In [3]:
# import packages & modules
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, classification_report, recall_score, f1_score, accuracy_score, precision_score
from sklearn.utils import shuffle
import lightgbm as lgb
import catboost as cgb
import catboost.datasets as cbd
import catboost.utils as cbu
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score
import warnings
import hyperopt
import sys
warnings.filterwarnings('ignore')

In [4]:
# seet seed - my favorite number, obviously
seed = 5
np.random.seed(seed)

In [5]:
# import training dataset
data = pd.read_csv('HW4 - training data.csv',header=0)

In [6]:
# sampling - I used undersampling of the majority class and found this to work
# better than oversampling the minority. I tested multiple ratios, but found that 
# a 1:4 ratio worked best for all of my models, using all of the minority class.

minority_class = np.array(data[data.adopter == 1].index)
majority_class = data[data.adopter == 0].index

# a 1:4 ratio using all of the minority means 1540 minority and 6160 majority
# let's randomly select the 6160 majority samples
# I did this without replacement to add more variety to the samples
random_majority_class = np.random.choice(majority_class, 6160, replace = False)
random_majority_class = np.array(random_majority_class)

# combine both classes now
sample = np.concatenate([minority_class, random_majority_class])

# get the data using the sample indices and shuffle the data before split
sample_data = data.iloc[sample,:]
sample_data = shuffle(sample_data)

# split into X and y
X = sample_data.iloc[:, data.columns != 'adopter']
y = sample_data.iloc[:, data.columns == 'adopter']

# split dataset for cross validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

print ("Number of train instances: {}".format(len(X_train)))
print ("Number of test instances: {}".format(len(X_test)))

Number of train instances: 5390
Number of test instances: 2310


## Light GBM 

#### Hyperparameter Optimization 

In [5]:
# I used this tutorial as a resource for tuning my parameters for LightGBM using Bayesian Optimization:
# https://medium.com/analytics-vidhya/hyperparameters-optimization-for-lightgbm-catboost-and-xgboost-regressors-using-bayesian-6e7c495947a9

In [6]:
# make train and test into a lightgbm Dataset
train_data = lgb.Dataset(X_train, label = y_train)
test_data = lgb.Dataset(X_test, label = y_test)

In [7]:
# define objective function and parameters
def hyp_lgbm(num_leaves, feature_fraction, bagging_fraction, max_depth, min_split_gain, min_child_weight):
        params = {'application':'regression','num_iterations': 5000,
                  'learning_rate':0.05, 'early_stopping_round':50,
                  'metric':'l1'} # Default parameters
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=5, seed=7, stratified=False, verbose_eval =None, metrics=['l1'])
        return -np.min(cv_result['l1-mean'])

In [8]:
# define parameters to search through
pds = {'num_leaves': (45, 60),
          'feature_fraction': (0.1, 0.9),
          'bagging_fraction': (0.8, 1),
          'max_depth': (9, 13 ),
          'min_split_gain': (0.001, 0.1),
          'min_child_weight': (30, 50)
          }

In [9]:
# define Bayesian Optimization function and maximize the output of objective function
# here, the sum of init_points and n_iter will equal the total number of rounds
optimizer = BayesianOptimization(hyp_lgbm,pds,random_state=7)
optimizer.maximize(init_points=5, n_iter=15)

|   iter    |  target   | baggin... | featur... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        | -0.255    |  0.8153   |  0.7239   |  10.75    |  44.47    |  0.09782  |  53.08    |
|  2        | -0.2651   |  0.9002   |  0.1576   |  10.07    |  40.0     |  0.06824  |  57.06    |
|  3        | -0.2649   |  0.8762   |  0.1527   |  10.15    |  48.19    |  0.02213  |  51.78    |
|  4        | -0.2678   |  0.9862   |  0.1199   |  11.4     |  49.0     |  0.0238   |  53.23    |
|  5        | -0.2644   |  0.9818   |  0.2065   |  11.09    |  45.01    |  0.06723  |  52.02    |
|  6        | -0.2525   |  0.8      |  0.9      |  13.0     |  30.0     |  0.001    |  45.0     |
|  7        | -0.2543   |  0.8115   |  0.8521   |  9.219    |  30.34    |  0.04937  |  59.59    |
|  8        | -0.2535   |  0.8138   |  0.7768   |  9.049    |  30.96    |  0.05473  |  45.16    |
|  9        | -0.254

In [10]:
# get optimized parameters
optimizer.max['params']

{'bagging_fraction': 0.8120618558202543,
 'feature_fraction': 0.8804924233810146,
 'max_depth': 12.267142340890745,
 'min_child_weight': 30.200387989636972,
 'min_split_gain': 0.031982756484661944,
 'num_leaves': 49.75859581031999}

#### Train Final Model 

In [11]:
# final parameters based on optimization
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'l1',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 46,
    'feature_fraction': 0.8804924233810146,
    'bagging_fraction': 0.8120618558202543,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 1,
    'max_depth': 12,
    'min_child_weight': 30.200387989636972,
    'min_split_gain': 0.031982756484661944
}

# train model
lgbm = lgb.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       verbose_eval=10,
                       num_boost_round=8000,
                       early_stopping_rounds=3500)

Training until validation scores don't improve for 3500 rounds
[10]	valid_0's l1: 0.338022
[20]	valid_0's l1: 0.344689
[30]	valid_0's l1: 0.344838
[40]	valid_0's l1: 0.342264
[50]	valid_0's l1: 0.34023
[60]	valid_0's l1: 0.336653
[70]	valid_0's l1: 0.333756
[80]	valid_0's l1: 0.331738
[90]	valid_0's l1: 0.329531
[100]	valid_0's l1: 0.326918
[110]	valid_0's l1: 0.324378
[120]	valid_0's l1: 0.321728
[130]	valid_0's l1: 0.31907
[140]	valid_0's l1: 0.317121
[150]	valid_0's l1: 0.315655
[160]	valid_0's l1: 0.314135
[170]	valid_0's l1: 0.312524
[180]	valid_0's l1: 0.310824
[190]	valid_0's l1: 0.309525
[200]	valid_0's l1: 0.308348
[210]	valid_0's l1: 0.307378
[220]	valid_0's l1: 0.306174
[230]	valid_0's l1: 0.305242
[240]	valid_0's l1: 0.304308
[250]	valid_0's l1: 0.300741
[260]	valid_0's l1: 0.297999
[270]	valid_0's l1: 0.298542
[280]	valid_0's l1: 0.298478
[290]	valid_0's l1: 0.298076
[300]	valid_0's l1: 0.296737
[310]	valid_0's l1: 0.295344
[320]	valid_0's l1: 0.294641
[330]	valid_0's l1: 

[5490]	valid_0's l1: 0.241603
[5500]	valid_0's l1: 0.241673
[5510]	valid_0's l1: 0.24188
[5520]	valid_0's l1: 0.242085
[5530]	valid_0's l1: 0.242177
[5540]	valid_0's l1: 0.24222
[5550]	valid_0's l1: 0.241869
[5560]	valid_0's l1: 0.241642
[5570]	valid_0's l1: 0.24203
[5580]	valid_0's l1: 0.242248
[5590]	valid_0's l1: 0.242171
[5600]	valid_0's l1: 0.242114
[5610]	valid_0's l1: 0.24214
[5620]	valid_0's l1: 0.2422
[5630]	valid_0's l1: 0.242095
[5640]	valid_0's l1: 0.242088
[5650]	valid_0's l1: 0.241864
[5660]	valid_0's l1: 0.241789
[5670]	valid_0's l1: 0.241748
[5680]	valid_0's l1: 0.241679
[5690]	valid_0's l1: 0.24176
[5700]	valid_0's l1: 0.241758
[5710]	valid_0's l1: 0.241403
[5720]	valid_0's l1: 0.241169
[5730]	valid_0's l1: 0.241428
[5740]	valid_0's l1: 0.241556
[5750]	valid_0's l1: 0.241546
[5760]	valid_0's l1: 0.24157
[5770]	valid_0's l1: 0.241799
[5780]	valid_0's l1: 0.2419
[5790]	valid_0's l1: 0.241916
[5800]	valid_0's l1: 0.241886
[5810]	valid_0's l1: 0.241829
[5820]	valid_0's l1:

In [12]:
# predict on X_test and classify based on a threshold of 0.6
y_pred = lgbm.predict(X_test)
y_pred[y_pred >= 0.6] = 1
y_pred[y_pred < 0.6] = 0

print ("F1: ", f1_score(y_pred, y_test))
print ("Recall: ", recall_score(y_pred, y_test))
print ("Prec.: ", precision_score(y_pred, y_test))
print ("Acc.: ", accuracy_score(y_pred, y_test))

F1:  0.41854934601664684
Recall:  0.46437994722955145
Prec.:  0.38095238095238093
Acc.:  0.7883116883116883


In [13]:
# now import test set without labels and make predictions
test = pd.read_csv('HW4 - test data.csv',header=0)

y_pred = lgbm.predict(test)

y_pred = pd.DataFrame({'prediction(adopter)': y_pred })
y_pred

,prediction(adopter)
0,0.005829
1,0.007001
2,0.141774
3,0.000410
4,0.000414
...,...
86676,0.000778
86677,0.843112
86678,0.036194
86679,0.319560


In [14]:
# save final predictions for ensembling later in Excel
np.savetxt("lightgbm.csv", y_pred , delimiter=",")

## Catboost

#### Hyperparameter Optimization 

In [15]:
# I used this tutorial as a resource for tuning my parameters for Catboost using Hyperopt:
# https://github.com/catboost/tutorials/blob/master/classification/classification_with_parameter_tuning_tutorial.ipynb

In [19]:
class UciAdultClassifierObjective(object):
    def __init__(self, dataset, const_params, fold_count):
        self._dataset = dataset
        self._const_params = const_params.copy()
        self._fold_count = fold_count
        self._evaluated_count = 0
        
    def _to_catboost_params(self, hyper_params):
        return {
            'learning_rate': hyper_params['learning_rate'],
            'depth': hyper_params['depth'],
            'l2_leaf_reg': hyper_params['l2_leaf_reg']}
    
    # hyperopt optimizes an objective using `__call__` method (e.g. by doing 
    # `foo(hyper_params)`), so we provide one
    def __call__(self, hyper_params):
        # join hyper-parameters provided by hyperopt with hyper-parameters 
        # provided by the user
        params = self._to_catboost_params(hyper_params)
        params.update(self._const_params)
        
        print('evaluating params={}'.format(params), file=sys.stdout)
        sys.stdout.flush()
        
        # we use cross-validation for objective evaluation, to avoid overfitting
        scores = cgb.cv(
            pool=self._dataset,
            params=params,
            fold_count=self._fold_count,
            partition_random_seed=20181224,
            verbose=False)
        
        # scores returns a dictionary with mean and std (per-fold) of metric 
        # value for each cv iteration, we choose minimal value of objective 
        # mean (though it will be better to choose minimal value among all folds)
        # because noise is additive
        max_mean_auc = np.max(scores['test-AUC-mean'])
        print('evaluated score={}'.format(max_mean_auc), file=sys.stdout)
        
        self._evaluated_count += 1
        print('evaluated {} times'.format(self._evaluated_count), file=sys.stdout)
           
        # negate because hyperopt minimizes the objective
        return {'loss': -max_mean_auc, 'status': hyperopt.STATUS_OK}

In [20]:
def find_best_hyper_params(dataset, const_params, max_evals=100):    
    # we are going to optimize these three parameters
    parameter_space = {
        'learning_rate': hyperopt.hp.uniform('learning_rate', 0.2, 1.0),
        'depth': hyperopt.hp.randint('depth', 7),
        'l2_leaf_reg': hyperopt.hp.uniform('l2_leaf_reg', 1, 10)}
    objective = UciAdultClassifierObjective(dataset=dataset, const_params=const_params, fold_count=6)
    trials = hyperopt.Trials()
    best = hyperopt.fmin(
        fn=objective,
        space=parameter_space,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals,
        rstate=np.random.RandomState(seed=20181224))
    return best

def train_best_model(X, y, const_params, max_evals=100, use_default=False):
    # convert pandas.DataFrame to catboost.Pool to avoid converting it on each 
    # iteration of hyper-parameters optimization
    dataset = cgb.Pool(X, y, cat_features=np.where(X.dtypes != np.float)[0])
    
    if use_default:
        # pretrained optimal parameters
        best = {
            'learning_rate': 0.4234185321620083, 
            'depth': 5, 
            'l2_leaf_reg': 9.464266235679002}
    else:
        best = find_best_hyper_params(dataset, const_params, max_evals=max_evals)
    
    # merge subset of hyper-parameters provided by hyperopt with hyper-parameters 
    # provided by the user
    hyper_params = best.copy()
    hyper_params.update(const_params)
    
    # drop `use_best_model` because we are going to use entire dataset for 
    # training of the final model
    hyper_params.pop('use_best_model', None)
    
    model = cgb.CatBoostClassifier(**hyper_params)
    model.fit(dataset, verbose=False)
    
    return model, hyper_params

In [21]:

# make it True if your want to use GPU for training
have_gpu = False
# skip hyper-parameter optimization and just use provided optimal parameters
use_optimal_pretrained_params = True
# number of iterations of hyper-parameter search
hyperopt_iterations = 30
const_params = dict({
    'task_type': 'GPU' if have_gpu else 'CPU',
    'loss_function': 'Logloss',
    'eval_metric': 'AUC', 
    'custom_metric': ['AUC'],
    'iterations': 100,
    'random_seed': 20181224})

model, params = train_best_model(
    X_train, y_train, 
    const_params, 
    max_evals=hyperopt_iterations, 
    use_default=use_optimal_pretrained_params)
print('best params are {}'.format(params), file=sys.stdout)

best params are {'learning_rate': 0.4234185321620083, 'depth': 5, 'l2_leaf_reg': 9.464266235679002, 'task_type': 'CPU', 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'custom_metric': ['AUC'], 'iterations': 100, 'random_seed': 20181224}


In [22]:
# predict on X_test and classify based on a threshold of 0.6
y_pred = model.predict(X_test)
y_pred[y_pred >= 0.6] = 1
y_pred[y_pred < 0.6] = 0

print ("F1: ", f1_score(y_pred, y_test))
print ("Recall: ", recall_score(y_pred, y_test))
print ("Prec.: ", precision_score(y_pred, y_test))
print ("Acc.: ", accuracy_score(y_pred, y_test))

F1:  0.3791208791208791
Recall:  0.518796992481203
Prec.:  0.2987012987012987
Acc.:  0.8043290043290043


In [23]:
# now import test set without labels and make predictions
test = pd.read_csv('HW4 - test data.csv',header=0)

y_pred = model.predict_proba(test)
y_pred

array([[0.81151465, 0.18848535],
       [0.9456155 , 0.0543845 ],
       [0.91688247, 0.08311753],
       ...,
       [0.95796661, 0.04203339],
       [0.89550243, 0.10449757],
       [0.41967292, 0.58032708]])

In [24]:
model.classes_
# predictions we want will be in the second column

array([0, 1], dtype=int64)

In [25]:
# save final predictions for ensembling later in Excel
np.savetxt("catboost.csv", y_pred , delimiter=",")

## XGBoost

#### Hyperparameter Optimization 

In [7]:
# I used this tutorial as a resource for tuning my parameters for XGBoost using Bayesian Optimization:
# https://medium.com/analytics-vidhya/hyperparameters-optimization-for-lightgbm-catboost-and-xgboost-regressors-using-bayesian-6e7c495947a9

In [8]:
# define R-Squared
def xgb_r2(preds, X_train):
    labels = X_train.get_label()
    return 'r2', r2_score(preds, labels)
  
X_train = xgb.DMatrix(X, y, feature_names=X.columns.values)

# define objective function
def hyp_xgb(max_depth, subsample, colsample_bytree,min_child_weight, gamma ):
    params = {
    'n_estimators': 300,
    'eta': 0.05,
    'objective': 'reg:linear',
    'eval_metric':'mae', # Optional --> Use eval_metric if you want to stop evaluation based on eval_metric 
    'silent': 1
     }
    params['max_depth'] = int(round(max_depth))
    params['subsample'] = max(min(subsample, 1), 0)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['min_child_weight'] = int(min_child_weight)
    params['gamma'] = max(gamma, 0)
    scores = xgb.cv(params, X_train, num_boost_round=1000,verbose_eval=False, early_stopping_rounds=10, feval=xgb_r2, maximize=True, nfold=5)
    return  scores['test-r2-mean'].iloc[-1]

In [9]:
# define parameters to search through
pds ={
  'min_child_weight':(14, 20),
  'gamma':(0, 5),
  'subsample':(0.5, 1),
  'colsample_bytree':(0.1, 1),
  'max_depth': (5, 10)
}

In [10]:
# define Bayesian Optimization function and maximize the output of objective function
# here, the sum of init_points and n_iter will equal the total number of rounds
optimizer = BayesianOptimization(hyp_xgb, pds, random_state=10)
optimizer.maximize(init_points=5, n_iter=15)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------
|  1        | -1.8      |  0.7942   |  0.1038   |  8.168    |  18.49    |  0.7493   |
|  2        | -2.602    |  0.3023   |  0.9903   |  8.803    |  15.01    |  0.5442   |
|  3        | -5.947    |  0.7168   |  4.767    |  5.02     |  17.07    |  0.9063   |
|  4        | -4.898    |  0.6513   |  3.609    |  6.459    |  19.51    |  0.8573   |
|  5        | -2.262    |  0.5883   |  0.7109   |  6.867    |  18.04    |  0.7209   |
|  6        | -2.189    |  0.1      |  0.0      |  10.0     |  17.44    |  1.0      |
|  7        | -2.07     |  0.1      |  0.0      |  10.0     |  20.0     |  0.5      |
|  8        | -2.155    |  0.1      |  0.0      |  6.743    |  20.0     |  0.5      |
|  9        | -1.94     |  1.0      |  0.0      |  5.0      |  14.0     |  0.5      |
|  10       | -1.75     |  1.0      |  0.0      |  6.7

In [11]:
# get optimized parameters
optimizer.max['params']

{'colsample_bytree': 1.0,
 'gamma': 0.0,
 'max_depth': 8.470384396725334,
 'min_child_weight': 20.0,
 'subsample': 1.0}

#### Train Final Model 

In [12]:
# redefine X_train, X_test, y_train, and y_test since we've used them previously
# split into X and y
X = sample_data.iloc[:, data.columns != 'adopter']
y = sample_data.iloc[:, data.columns == 'adopter']

# split dataset for cross validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

print ("Number of train instances: {}".format(len(X_train)))
print ("Number of test instances: {}".format(len(X_test)))

Number of train instances: 5390
Number of test instances: 2310


In [13]:
# create train and test xgb matrices
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

num_rounds = 100

params = {
    'max_depth':7,
    'n_estimators': 5000,
    'gamma': 0,
    'scale_pos_weight': 1,
    'eta': 0.1,
    'reg_alpha': 0.05,
    'objective': 'binary:logistic',
    'seed': 557,
    'colsample_bytree': 1,
    'min_child_weight': 17.90432554809831,
    'subsample': 1
}

test_train_split = [(dtest, 'test'), (dtrain, 'train')]

boost = xgb.train(params,
                 dtrain,
                 num_rounds, 
                 test_train_split)

[0]	test-error:0.20217	train-error:0.18126
[1]	test-error:0.19870	train-error:0.17681
[2]	test-error:0.19913	train-error:0.17403
[3]	test-error:0.19827	train-error:0.17180
[4]	test-error:0.19351	train-error:0.16865
[5]	test-error:0.19524	train-error:0.17069
[6]	test-error:0.19784	train-error:0.16883
[7]	test-error:0.19567	train-error:0.16846
[8]	test-error:0.19654	train-error:0.16883
[9]	test-error:0.20000	train-error:0.16475
[10]	test-error:0.20043	train-error:0.16289
[11]	test-error:0.19437	train-error:0.16215
[12]	test-error:0.19394	train-error:0.16197
[13]	test-error:0.19178	train-error:0.16197
[14]	test-error:0.19394	train-error:0.16085
[15]	test-error:0.19351	train-error:0.15993
[16]	test-error:0.19610	train-error:0.15733
[17]	test-error:0.19610	train-error:0.15788
[18]	test-error:0.19481	train-error:0.15677
[19]	test-error:0.19264	train-error:0.15547
[20]	test-error:0.19307	train-error:0.15603
[21]	test-error:0.19610	train-error:0.15621
[22]	test-error:0.19221	train-error:0.1545

In [14]:
# predict on dtest and classify based on a threshold of 0.6
y_pred = boost.predict(dtest)
y_pred[y_pred >= 0.6] = 1
y_pred[y_pred < 0.6] = 0

print (accuracy_score(y_pred, y_test))
print (f1_score(y_pred, y_test))
print (recall_score(y_pred, y_test))
print (precision_score(y_pred, y_test))

0.8069264069264069
0.28525641025641024
0.5493827160493827
0.19264069264069264


In [15]:
# now import test set without labels and make predictions
test = pd.read_csv('HW4 - test data.csv',header=0)
test = xgb.DMatrix(test)

y_pred = boost.predict(test)

y_pred = pd.DataFrame({'prediction(adopter)': y_pred })
y_pred

,prediction(adopter)
0,0.129029
1,0.025890
2,0.114438
3,0.016514
4,0.035663
...,...
86676,0.015777
86677,0.555487
86678,0.102279
86679,0.139987


In [43]:
# save final predictions for ensembling later in Excel
np.savetxt("xgboost.csv", y_pred , delimiter=",")

In [44]:
# I originally did each model in separate Python notebooks, so that's why I export to Excel to combine them
# I averaged the predictions in Excel, then classified each one using a threshold of 0.6